# Renumber Ligand Atoms' Residue Numbers

## Purpose
See how BioPandas can be used to renumber residue numbers for ligand atoms.

## Methodology
Quickly describe assumptions and processing steps.

## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- todo 1;
- todo 2;
- todo 3.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
from biopandas.pdb import PandasPdb

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

## Local library import
We import all the required local libraries libraries

In [2]:
import os
if os.path.basename(os.getcwd()) == "notebooks" or os.path.basename(os.getcwd()) == "notebook":
    os.chdir('..')
    os.getcwd()

In [3]:
# Include local library paths
import sys
# sys.path.append('path/to/local/lib') # uncomment and fill to import local libraries

# Import local libraries

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [4]:
PDB_FILEPATH = "pdb/external/1g6n.1.pdb"

# Data import
We retrieve all the required data for the analysis.

In [5]:
protein_pdb = PandasPdb().read_pdb(PDB_FILEPATH)

# Data processing
Put here the core of the notebook. Feel free to further split this section into subsections.

In [6]:
ligand_pdb = protein_pdb.df['HETATM']
# Rename atom name
ligand_pdb['atom_name'] = "LIG"
# Renumber residue numbers
ligand_pdb['residue_number'] = ligand_pdb['residue_number'] + np.arange(ligand_pdb['residue_number'].shape[0])

In [7]:
protein_pdb.df['HETATM']

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,blank_3,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,HETATM,3173,,LIG,,CMP,,A,621,,,40.558,42.062,52.021,1.0,16.49,,,P,NaN,3158
1,HETATM,3174,,LIG,,CMP,,A,623,,,40.284,41.032,50.963,1.0,13.87,,,O,NaN,3159
2,HETATM,3175,,LIG,,CMP,,A,625,,,40.660,41.690,53.413,1.0,16.14,,,O,NaN,3160
3,HETATM,3176,,LIG,,CMP,,A,627,,,41.852,42.872,51.531,1.0,15.03,,,O,NaN,3161
4,HETATM,3177,,LIG,,CMP,,A,629,,,41.764,43.600,50.298,1.0,13.39,,,C,NaN,3162
5,HETATM,3178,,LIG,,CMP,,A,631,,,40.695,44.641,50.513,1.0,14.05,,,C,NaN,3163
6,HETATM,3179,,LIG,,CMP,,A,633,,,40.334,45.393,49.361,1.0,13.84,,,O,NaN,3164
7,HETATM,3180,,LIG,,CMP,,A,635,,,39.400,43.962,50.721,1.0,13.89,,,C,NaN,3165
8,HETATM,3181,,LIG,,CMP,,A,637,,,39.511,43.314,52.032,1.0,14.80,,,O,NaN,3166
9,HETATM,3182,,LIG,,CMP,,A,639,,,38.310,45.089,50.723,1.0,13.90,,,C,NaN,3167


In [8]:
# Inter-connect ligand atoms in -spcust format
from itertools import combinations
ligand_residues = [tuple(item) for item in np.vstack((ligand_pdb['residue_number'], ligand_pdb['chain_id'])).T]
ligand_springs = list(combinations(ligand_residues, 2))

In [10]:
ligand_springs[:5]

[((621, 'A'), (623, 'A')),
 ((621, 'A'), (625, 'A')),
 ((621, 'A'), (627, 'A')),
 ((621, 'A'), (629, 'A')),
 ((621, 'A'), (631, 'A'))]

In [11]:
# Write ligand-protein (amino acid  and ligand atoms) interactions 
protein_ligand_springs =[((71, 'A'), (631, 'A')),\
                         ((72, 'A'), (631, 'A')),\
                         ((82, 'A'), (623, 'A')),\
                         ((83, 'A'), (622, 'A')),\
                         ((127, 'A'), (638, 'A')),\
                         ((128, 'B'), (638, 'A'))]

In [13]:
# Write spfile for GENENMM's -spcust flag input
def write_spfile(data , output_dir="misc/"):
    """ Writes spfile from a list of tuples representing
        springs between two residues.
    """
    spfile_line_format = " {:4d} {:1s} {:4d} {:1s}\n"
    
    spfile_lines = []
    for line in data:
        residue_number_1 = line[0][0]
        chain_id_1 = line[0][1]
        residue_number_2 = line[1][0]
        chain_id_2 = line[1][1]

        spfile_line = spfile_line_format.format(residue_number_1, chain_id_1,\
                                                residue_number_2, chain_id_2)
        spfile_lines.append(spfile_line)
    
    spfile_output_filepath = os.path.join(output_dir, "fix.springs")  
    with open(spfile_output_filepath, 'w') as spfile:
        spfile.writelines(spfile_lines)
        
    return None

spring_data = ligand_springs + protein_ligand_springs
write_spfile(spring_data)

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2